Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.  The defendant is known to have this rare blood type.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

The prosecuter is confounding the probaility of any random person having the blood type with the probaility of guilt given that the defendent has the blood type. To find the latter you must have soem prior probability of guilt which will inform the influence of the the blood evidence on the conditional probability of guilt. 

b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument (HINT: What happens to the prior in this case if there is *other* evidence presented)?

Again the error comes from ignoring the influence of the prior P(G=1).  If the blood type is the only eviedence presented and otherwise there is no reason to believe the defendent to me more or less guilty than any other random person we could say that P(G=1)=1/800000 in which case P(G=1|B=1)=1/8000. However, if any other evidence increases or decreases the probaility  of guilt the influence of the blood evidence must also be reconsidered. 

c. Suppose that forensic analysis tells us that that the blood test has 98% sensitivity (true positive rate) and a 1% false positive rate.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene *and* that there is no other evidence.  

We want to find the probability of guilt (G) given a a positive test result on the blood at the scene (T), or P(G=1|T=1).  Using bayes we get...

P(G=1|T=1)= P(T=1|G=1)P(G=1)/(P(T=1|G=1)P(G=1) + P(T=1|G=0)P(G=0))

We know that if the defendent is guity his blood must match the blood at the scene and that given the blood is in fact the rare type that the probabilty of getting a positve test result is 0.98. We also know that, given access to the crime scene, the prior porability of guilt is 0.2. We can now fill in most of the needed probabilities.

P(G=1|T=1)= (0.8)(0.2)/((0.98)(0.2) + P(T=1|G=0)(0.8))

To find the last needed probaility we need to futher condition on the proability that the blood at the scene is the rare type is the defendent is not guilty.

P(T=1|G=0) = P(T=1|G=0,B=1)P(B=1|G=0) + P(T=1|G=0,B=0)P(B=0|G=0)

If the defendent is not guilty the probaility of the blood being the rare type can be assumed to by that of it existing in any random person in the population (although it really should by the probability of it being in one of the other 4 people with acces to the crime scene).

P(T=1|G=0) = (0.98)(0.01) + (0.01)(0.99)

Thus, 

P(G=1|T=1)= (0.8)(0.2)/((0.98)(0.2) + P(0.0197)(0.8)) = 0.9256

There is a 92.6% chance the defendent is guilty.

Problem 2: Naive Bayes.

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|X=x) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|X=x) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [5]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [11]:
#What is the prior P(Y=ham) ?
ham_prior = np.sum(training_labels) / len(training_labels)

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = word_dictionary[key][1] / (word_dictionary[key][0] + word_dictionary[key][1])


Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [14]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = 1.0 
    posterior_spam = 1.0 
    spam_prior = 1 - ham_prior
    # Loop over all the words in each message
    for w in m:
        # Skips over word not found in training data #! What is the purpose of this try/except handler?
        try:
            posterior_ham *= ham_likelihood[w] * ham_prior / (ham_likelihood[w] * ham_prior + (1 - ham_likelihood[w]) * spam_prior)
            posterior_spam *= (1 - ham_likelihood[w]) * spam_prior / (ham_likelihood[w] * ham_prior + (1 - ham_likelihood[w]) * spam_prior)
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [25]:
    correct = 0
    total = 0
    predicted_labels = np.zeros(len(test_lines),dtype=int)
    # determine whether the spam or ham postirior is greater and label accordingly #
    if(posteriors[i,1] >= posteriors[i,0]):
        predicted_labels[i] = 1
    else:
        predicted_labels[i] = 0
    
    if(predicted_labels[i] == test_labels[i]):
        correct += 1
    total +=1
    
percent_correct = int(correct/total * 100)

print("This tool is " + str(percent_correct)+ "% accurate.")

This tool is 100% accurate.
